In [2]:
# Variance reductino using Control Variates

import numpy as np
from scipy import stats
 
## BSM closed-form solution (for call) ##
def bsm_call_value(S0, K, T, r, sigma):
    d1= (np.log(S0/K)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2= (np.log(S0/K)+(r-0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    call_value = (S0*stats.norm.cdf(d1,0.0,1.0) - K*np.exp(-r*T)*stats.norm.cdf(d2,0.0,1.0))
    return call_value

    
np.random.seed(0)
N=int(1e6)
 
S0=100 ; K=105 ; T=2 ; r=0.05 ; sig=0.2

M = 8  
dt = T/M

mu = np.zeros(M)
A = np.zeros((M,M))
for i in range(M):
    for j in range(M):
        if i>=j:
            A[i,j] = np.sqrt(dt)

mu = np.zeros(M)
eye_M = np.eye(M)    
Z = np.random.multivariate_normal(mu,eye_M,N)      
W = np.dot(A,Z.T)
W=W.T
S_t = np.zeros((N,M))
t = np.zeros(M)
for j in range(M):
    t[j] = dt*(j+1)
    S_t[:,j] = S0*np.exp((r-sig*sig/2)*t[j]+sig*W[:,j])

S_t_bar=np.zeros(N) # 산술 평균
S_t_gbar=np.zeros(N) # geometric 평균

for k in range(N):
    S_t_bar[k]=np.mean(S_t[k,:]) # 산술평균 계산
    S_t_gbar[k]=(S0*np.prod(S_t[k,:]))**(1/(M+1)) # 기하평균 계산

h_T = (S_t_bar-K)*(S_t_bar>K) # 일반적 asian call option pay-off
CV_T = (S_t_gbar-K)*(S_t_gbar>K) # geometric asian call option pay-off
h_0 = np.exp(-r*T)*h_T
CV_0 = np.exp(-r*T)*CV_T  # 부산물

## closed-form solution of a version of Asian call (= geometric asian call)
# control Variagte(IV) 참조
sig_z = sig*np.sqrt((2*M+1)/(6*M+6))
rho = 0.5*((r-sig*sig/2)+sig_z*sig_z)
E_CV_0 = np.exp((rho-r)*T)*bsm_call_value(S0, K, T, rho, sig_z) #


## control variate method
cov_mat = np.cov(h_0,CV_0)
c_star = - cov_mat[0,1]/cov_mat[1,1] # C*

h_0_c = h_0 + c_star*(CV_0-E_CV_0)

h_0_e = np.mean(h_0_c) ; h_0_std = np.std(h_0_c)

print("h_0 =",h_0_e) 
print("95% confidence interval =",[h_0_e-1.96*h_0_std/np.sqrt(N),h_0_e+1.96*h_0_std/np.sqrt(N)])

h_0 = 7.2293873908272
95% confidence interval = [7.228221305965206, 7.230553475689194]


In [ ]:
# Variance reductino using Antithetic variation

